# NIHCXR Synthetic Drift - Categorical Shift

## Load Libraries

In [ ]:
from drift_detection.drift_detector import (
    Detector,
    Experimenter,
    Reductor,
    SyntheticShiftApplicator,
    TSTester,
    plot_drift_samples_pval,
)

# from drift_detection.datasets.utils import load_dataset

## Query Data

In [ ]:
load_dataset = callable()
dataset, metadata, metadata_mapping = load_dataset(
    "nihcxr", "../../datasets/configs/nihcxr.yaml"
)

type(dataset)

## Initalize Reductor, Tester & Detector

In [ ]:
reductor = Reductor(
    dr_method="TAE_txrv_CNN",
)

tester = TSTester(
    tester_method="mmd",
)


detector = Detector(
    reductor=reductor,
    tester=tester,
)

detector.fit(dataset, progress=False)

## Setup Baseline Experiment

In [ ]:
baseline_experiment = Experimenter(
    "sensitivity_test",
    detector=detector,
)

## Setup Drift Experiment (Categorical Shift)

In [ ]:
shiftapplicator = SyntheticShiftApplicator(
    shift_type="categorical_shift",
    metadata=dataset.dataset.csv.iloc[dataset.indices, :],
    categorical_column="Patient Gender",
    target_category="M",
)

drift_experiment = Experimenter(
    "sensitivity_test",
    detector=detector,
    shiftapplicator=shiftapplicator,
)

## Run Experiments & Gather Results

In [ ]:
baseline_results = baseline_experiment.run(dataset)
drift_results = drift_experiment.run(dataset)

results = {"baseline": baseline_results, "experiment": drift_results}

## Plot Experimental Results

In [ ]:
plot_drift_samples_pval(results, 0.05)